## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [2]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "Travelers Championship"
tournament_date = "6/22/2025"  # Ending date of tournament
old_course = "TPC River Highlands"
tournament_id = "R2025034"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Rocket Classic"
new_ending_date = "6/29/2025"
new_course = "Detroit Golf Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [3]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025034 (Travelers Championship), year: 20250
✅ 72 new rows added for 'Travelers Championship'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
32,2025,2025-06-22,R2025034,Travelers Championship,TPC River Highlands,Keegan Bradley,1,1,-6,E,-7,-2,"$3,600,000.00",700.000
52,2025,2025-06-22,R2025034,Travelers Championship,TPC River Highlands,Russell Henley,T2,2,-2,-2,-9,-1,"$1,760,000.00",375.000
67,2025,2025-06-22,R2025034,Travelers Championship,TPC River Highlands,Tommy Fleetwood,T2,2,-4,-5,-7,+2,"$1,760,000.00",375.000
27,2025,2025-06-22,R2025034,Travelers Championship,TPC River Highlands,Jason Day,T4,4,-4,-4,-3,-2,"$880,000.00",312.500
21,2025,2025-06-22,R2025034,Travelers Championship,TPC River Highlands,Harris English,T4,4,-3,-2,-3,-5,"$880,000.00",312.500


#### Stats

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,612,.1930,2025
1,Aaron Baddeley,160.0,-1.022,168.0,-1.292,137.0,-0.265,4.0,0.536,153.0,...,289.6,113.0,57.29%,164.0,60.48%,81.0,60.24%,466,.2814,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,361,.3893,2025
3,Aaron Rai,25.0,0.769,22.0,0.407,50.0,0.280,70.0,0.082,112.0,...,287.3,1.0,74.04%,13.0,69.44%,103.0,58.77%,33,2.7049,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828,.1167,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 5 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
312,2019,Detroit Golf Club,524,Rocket Mortgage Classic,2019-06-30
156,2020,Detroit Golf Club,524,Rocket Mortgage Classic,2020-07-05
0,2021,Detroit Golf Club,524,Rocket Mortgage Classic,2021-07-04
468,2023,Detroit Golf Club,R2023524,Rocket Mortgage Classic,2023-07-02
624,2024,Detroit Golf Club,R2024524,Rocket Mortgage Classic,2024-06-30


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2019-06-30 — Rocket Mortgage Classic (492 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,8,273.88,53.3,18.26,0,2019-06-30,Rocket Mortgage Classic
1,Aaron Wise,14,9,301.72,64.3,21.55,1,2019-06-30,Rocket Mortgage Classic
2,Abraham Ancer,17,14,462.77,82.4,27.22,6,2019-06-30,Rocket Mortgage Classic



📆 2020-07-05 — Rocket Mortgage Classic (396 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,11,5,95.40,45.5,8.67,0,2020-07-05,Rocket Mortgage Classic
1,Aaron Wise,12,4,110.75,33.3,9.23,1,2020-07-05,Rocket Mortgage Classic
2,Abraham Ancer,11,10,839.66,90.9,76.33,10,2020-07-05,Rocket Mortgage Classic



📆 2021-07-04 — Rocket Mortgage Classic (485 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,10,2,18.08,20.0,1.81,0,2021-07-04,Rocket Mortgage Classic
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2021-07-04,Rocket Mortgage Classic
2,Aaron Terrazas,1,0,0.00,0.0,0.00,0,2021-07-04,Rocket Mortgage Classic



📆 2023-07-02 — Rocket Mortgage Classic (542 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,325.286,71.4,23.23,0,2023-07-02,Rocket Mortgage Classic
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-07-02,Rocket Mortgage Classic
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-07-02,Rocket Mortgage Classic



📆 2024-06-30 — Rocket Mortgage Classic (503 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,13,9,181.975,69.2,14.00,0,2024-06-30,Rocket Mortgage Classic
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-06-30,Rocket Mortgage Classic
2,Aaron Rai,16,13,544.507,81.2,34.03,7,2024-06-30,Rocket Mortgage Classic


### Recent Form

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2019-06-30 — Rocket Mortgage Classic (492 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Jazz Janewattananond,1,14.0,20.20,2019-06-30,Rocket Mortgage Classic
1,Rory McIlroy,10,15.2,6.34,2019-06-30,Rocket Mortgage Classic
2,Dustin Johnson,10,15.6,6.51,2019-06-30,Rocket Mortgage Classic



📆 2020-07-05 — Rocket Mortgage Classic (396 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Tyrrell Hatton,3,3.3,2.38,2020-07-05,Rocket Mortgage Classic
1,Patrick Cantlay,5,9.0,5.02,2020-07-05,Rocket Mortgage Classic
2,Bryson DeChambeau,7,11.7,5.63,2020-07-05,Rocket Mortgage Classic



📆 2021-07-04 — Rocket Mortgage Classic (485 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Guido Migliozzi,2,8.5,7.74,2021-07-04,Rocket Mortgage Classic
1,Dawie van der Walt,1,20.0,28.85,2021-07-04,Rocket Mortgage Classic
2,Xander Schauffele,12,22.3,8.69,2021-07-04,Rocket Mortgage Classic



📆 2023-07-02 — Rocket Mortgage Classic (542 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-07-02,Rocket Mortgage Classic
1,Brooks Koepka,3,6.7,4.83,2023-07-02,Rocket Mortgage Classic
2,Scottie Scheffler,18,7.6,2.58,2023-07-02,Rocket Mortgage Classic



📆 2024-06-30 — Rocket Mortgage Classic (503 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,3,3.0,2.16,2024-06-30,Rocket Mortgage Classic
1,Scottie Scheffler,16,6.3,2.22,2024-06-30,Rocket Mortgage Classic
2,Xander Schauffele,16,12.9,4.55,2024-06-30,Rocket Mortgage Classic


### Course History

In [8]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Rocket Mortgage Classic on 2020-07-05


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Wise,1,35.0,50.49,2020-07-05,Detroit Golf Club,Rocket Mortgage Classic
1,Adam Long,1,90.0,129.84,2020-07-05,Detroit Golf Club,Rocket Mortgage Classic
2,Adam Schenk,1,42.0,60.59,2020-07-05,Detroit Golf Club,Rocket Mortgage Classic



🏌️‍♂️ Course history for Rocket Mortgage Classic on 2021-07-04


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2021-07-04,Detroit Golf Club,Rocket Mortgage Classic
1,Aaron Wise,2,62.5,56.89,2021-07-04,Detroit Golf Club,Rocket Mortgage Classic
2,Adam Hadwin,1,4.0,5.77,2021-07-04,Detroit Golf Club,Rocket Mortgage Classic



🏌️‍♂️ Course history for Rocket Mortgage Classic on 2023-07-02


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,2,65.5,59.62,2023-07-02,Detroit Golf Club,Rocket Mortgage Classic
1,Aaron Wise,2,62.5,56.89,2023-07-02,Detroit Golf Club,Rocket Mortgage Classic
2,Adam Hadwin,2,47.0,42.78,2023-07-02,Detroit Golf Club,Rocket Mortgage Classic



🏌️‍♂️ Course history for Rocket Mortgage Classic on 2024-06-30


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,73.7,53.16,2024-06-30,Detroit Golf Club,Rocket Mortgage Classic
1,Aaron Rai,1,9.0,12.98,2024-06-30,Detroit Golf Club,Rocket Mortgage Classic
2,Aaron Wise,2,62.5,56.89,2024-06-30,Detroit Golf Club,Rocket Mortgage Classic


In [12]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

ℹ️ Empty course_hist for date 2019-06-30


## Training Dataset

In [13]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Aaron Wise,T35,35,56.0,0.475,14.0,0.533,129.0,-0.085,99.0,0.026,132.0,-0.146,7.0,24.8,27.0,3.01,116.0,4.04,23.0,4.54,24.0,140.0,33.0,302.6,107.0,61.75,38.0,68.6,186.0,52.05,81.0,1.77,40.0,64.3,301.72,21.55,1.0,49.7,18.35,NaN,NaN,0
1,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Long,CUT,90,121.0,-0.049,118.0,-0.036,78.0,0.138,145.0,-0.151,170.0,-0.369,178.0,18.86,165.0,3.10,153.0,4.06,144.0,4.67,47.0,158.0,112.0,292.0,46.0,66.51,134.0,65.28,126.0,58.74,127.0,1.20,150.0,42.1,671.78,35.36,1.0,66.8,22.30,NaN,NaN,0
2,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Schenk,T42,42,94.0,0.146,91.0,0.080,76.0,0.142,127.0,-0.075,33.0,0.341,45.0,22.67,41.0,3.02,61.0,4.01,60.0,4.59,55.0,162.0,47.0,300.8,115.0,61.27,36.0,68.61,74.0,60.71,219.0,0.78,NaN,61.9,347.58,16.55,0.0,56.1,18.15,NaN,NaN,0
3,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Svensson,CUT,90,17.0,0.981,58.0,0.213,23.0,0.509,33.0,0.259,183.0,-0.602,77.0,21.64,70.0,3.04,61.0,4.01,75.0,4.61,106.0,193.0,130.0,289.8,63.0,65.24,19.0,69.4,75.0,60.7,434.0,0.42,NaN,40.0,87.81,5.85,0.0,70.1,25.28,NaN,NaN,0
4,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Alex Cejka,CUT,90,153.0,-0.362,133.0,-0.092,114.0,-0.014,169.0,-0.257,174.0,-0.482,185.0,17.44,3.0,2.97,182.0,4.09,185.0,4.77,150.0,222.0,183.0,278.0,39.0,67.19,156.0,64.44,99.0,59.69,631.0,0.26,NaN,38.5,28.59,2.20,1.0,77.5,29.37,NaN,NaN,0
5,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Alex Prugh,CUT,90,75.0,0.278,27.0,0.379,59.0,0.222,179.0,-0.323,167.0,-0.344,148.0,20.19,165.0,3.10,79.0,4.02,70.0,4.60,5.0,98.0,27.0,303.7,71.0,64.08,3.0,71.44,183.0,52.88,435.0,0.42,150.0,56.2,158.31,9.89,2.0,63.4,22.38,NaN,NaN,0
6,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Anders Albertson,T64,64,133.0,-0.213,158.0,-0.265,68.0,0.165,135.0,-0.113,163.0,-0.307,146.0,20.23,7.0,2.98,139.0,4.05,154.0,4.68,159.0,235.0,105.0,293.2,130.0,60.11,61.0,67.71,38.0,62.37,395.0,0.46,NaN,37.5,163.61,10.23,0.0,69.6,24.57,NaN,NaN,0
7,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Andrew Landry,T59,59,95.0,0.142,65.0,0.185,64.0,0.200,166.0,-0.244,133.0,-0.153,83.0,21.47,70.0,3.04,79.0,4.02,144.0,4.67,59.0,165.0,159.0,286.8,6.0,71.72,81.0,67.12,87.0,60.27,146.0,1.07,NaN,55.6,140.12,7.78,1.0,65.0,22.08,NaN,NaN,0
8,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Anirban Lahiri,T42,42,176.0,-0.869,173.0,-0.431,155.0,-0.298,141.0,-0.139,68.0,0.116,154.0,20.05,52.0,3.03,153.0,4.06,181.0,4.75,173.0,257.0,125.0,290.2,132.0,60.08,177.0,62.96,104.0,59.57,294.0,0.61,250.0,47.1,122.72,7.22,0.0,70.6,24.43,NaN,NaN,0
9,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Austen Truslow,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,928.0,0.14,NaN,0.0,0.00,0.00,0.0,90.0,129.84,NaN,NaN,0


In [14]:
# Play with different years to see if I notice anything wrong (check Odds)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
624,2024,2024-06-30,Rocket Mortgage Classic,Detroit Golf Club,Aaron Baddeley,T72,72,167.0,-0.982,175.0,-1.009,153.0,-0.375,7.0,0.402,19.0,0.470,155.0,20.20%,11.0,2.99,141.0,4.05,158.0,4.67,175.0,316.0,171.0,284.8,145.0,56.21%,173.0,62.17%,7.0,65.97%,299.0,0.4698,300.0,69.2,181.975,14.00,0.0,54.8,20.76,73.7,53.16,0
625,2024,2024-06-30,Rocket Mortgage Classic,Detroit Golf Club,Aaron Rai,T2,2,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,30.0,81.2,544.507,34.03,7.0,39.8,14.05,9.0,12.98,1
626,2024,2024-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Schenk,CUT,90,146.0,-0.421,53.0,0.212,166.0,-0.520,130.0,-0.114,85.0,0.051,132.0,21.24%,152.0,3.09,131.0,4.04,116.0,4.61,102.0,180.0,82.0,302.4,98.0,59.93%,149.0,64.50%,74.0,60.50%,82.0,1.4461,80.0,65.2,577.709,25.12,1.0,52.8,16.61,30.0,18.64,0
627,2024,2024-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Svensson,CUT,90,33.0,0.592,99.0,0.019,28.0,0.380,39.0,0.193,164.0,-0.453,153.0,20.41%,142.0,3.08,85.0,4.01,121.0,4.62,62.0,158.0,118.0,298.3,40.0,64.18%,88.0,67.13%,60.0,61.39%,112.0,1.1671,50.0,80.0,487.341,19.49,11.0,46.4,14.24,65.0,59.17,0
628,2024,2024-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adrien Dumont de Chassart,CUT,90,169.0,-1.094,157.0,-0.381,157.0,-0.421,157.0,-0.292,141.0,-0.244,141.0,21.06%,142.0,3.08,141.0,4.05,116.0,4.61,154.0,229.0,69.0,303.9,160.0,54.73%,104.0,66.23%,171.0,53.47%,263.0,0.5384,300.0,45.5,37.255,3.39,2.0,76.1,30.62,NaN,NaN,0


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [15]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Aaron Wise,T35,35,56.0,0.475,14.0,0.533,129.0,-0.085,99.0,0.026,132.0,-0.146,7.0,24.8,27.0,3.01,116.0,4.04,23.0,4.54,24.0,140.0,33.0,302.6,107.0,61.75,38.0,68.6,186.0,52.05,81.0,1.77,40.0,64.3,301.72,21.55,1.0,49.7,18.35,NaN,NaN,0
1,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Long,CUT,90,121.0,-0.049,118.0,-0.036,78.0,0.138,145.0,-0.151,170.0,-0.369,178.0,18.86,165.0,3.10,153.0,4.06,144.0,4.67,47.0,158.0,112.0,292.0,46.0,66.51,134.0,65.28,126.0,58.74,127.0,1.20,150.0,42.1,671.78,35.36,1.0,66.8,22.30,NaN,NaN,0
2,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Schenk,T42,42,94.0,0.146,91.0,0.080,76.0,0.142,127.0,-0.075,33.0,0.341,45.0,22.67,41.0,3.02,61.0,4.01,60.0,4.59,55.0,162.0,47.0,300.8,115.0,61.27,36.0,68.61,74.0,60.71,219.0,0.78,1000.0,61.9,347.58,16.55,0.0,56.1,18.15,NaN,NaN,0
3,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Svensson,CUT,90,17.0,0.981,58.0,0.213,23.0,0.509,33.0,0.259,183.0,-0.602,77.0,21.64,70.0,3.04,61.0,4.01,75.0,4.61,106.0,193.0,130.0,289.8,63.0,65.24,19.0,69.4,75.0,60.7,434.0,0.42,1000.0,40.0,87.81,5.85,0.0,70.1,25.28,NaN,NaN,0
4,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Alex Cejka,CUT,90,153.0,-0.362,133.0,-0.092,114.0,-0.014,169.0,-0.257,174.0,-0.482,185.0,17.44,3.0,2.97,182.0,4.09,185.0,4.77,150.0,222.0,183.0,278.0,39.0,67.19,156.0,64.44,99.0,59.69,631.0,0.26,1000.0,38.5,28.59,2.20,1.0,77.5,29.37,NaN,NaN,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [16]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Aaron Wise,T35,35,56.0,0.475,14.0,0.533,129.0,-0.085,99.0,0.026,132.0,-0.146,7.0,24.80,27.0,3.01,116.0,4.04,23.0,4.54,24.0,140.0,33.0,302.6,107.0,61.75,38.0,68.60,186.0,52.05,81.0,1.77,40.0,64.3,301.72,21.55,1.0,49.7,18.35,60.331658,69.656206,0
1,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Long,CUT,90,121.0,-0.049,118.0,-0.036,78.0,0.138,145.0,-0.151,170.0,-0.369,178.0,18.86,165.0,3.10,153.0,4.06,144.0,4.67,47.0,158.0,112.0,292.0,46.0,66.51,134.0,65.28,126.0,58.74,127.0,1.20,150.0,42.1,671.78,35.36,1.0,66.8,22.30,60.331658,69.656206,0
2,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Schenk,T42,42,94.0,0.146,91.0,0.080,76.0,0.142,127.0,-0.075,33.0,0.341,45.0,22.67,41.0,3.02,61.0,4.01,60.0,4.59,55.0,162.0,47.0,300.8,115.0,61.27,36.0,68.61,74.0,60.71,219.0,0.78,1000.0,61.9,347.58,16.55,0.0,56.1,18.15,60.331658,69.656206,0
3,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Adam Svensson,CUT,90,17.0,0.981,58.0,0.213,23.0,0.509,33.0,0.259,183.0,-0.602,77.0,21.64,70.0,3.04,61.0,4.01,75.0,4.61,106.0,193.0,130.0,289.8,63.0,65.24,19.0,69.40,75.0,60.70,434.0,0.42,1000.0,40.0,87.81,5.85,0.0,70.1,25.28,60.331658,69.656206,0
4,2019,2019-06-30,Rocket Mortgage Classic,Detroit Golf Club,Alex Cejka,CUT,90,153.0,-0.362,133.0,-0.092,114.0,-0.014,169.0,-0.257,174.0,-0.482,185.0,17.44,3.0,2.97,182.0,4.09,185.0,4.77,150.0,222.0,183.0,278.0,39.0,67.19,156.0,64.44,99.0,59.69,631.0,0.26,1000.0,38.5,28.59,2.20,1.0,77.5,29.37,60.331658,69.656206,0


In [17]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [18]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Collin Morikawa,10900
1,Keegan Bradley,10400
2,Patrick Cantlay,10200
3,Ben Griffin,9800
4,Hideki Matsuyama,9600
5,Harry Hall,9500
6,Cameron Young,9300
7,Si Woo Kim,9200
8,Max Greyserman,9100
9,Luke Clanton,9000


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Rocket Classic,Collin Morikawa,12/1,12.0
2,2025,Rocket Classic,Patrick Cantlay,14/1,14.0
3,2025,Rocket Classic,Keegan Bradley,18/1,18.0
4,2025,Rocket Classic,Ben Griffin,20/1,20.0
5,2025,Rocket Classic,Hideki Matsuyama,25/1,25.0
6,2025,Rocket Classic,Cameron Young,25/1,25.0
7,2025,Rocket Classic,Si Woo Kim,35/1,35.0
8,2025,Rocket Classic,Tony Finau,40/1,40.0
9,2025,Rocket Classic,Rickie Fowler,40/1,40.0
10,2025,Rocket Classic,Luke Clanton,40/1,40.0


## Cut Percentage and FedEx Points

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-06-29,Rocket Classic
1,Aaron Baddeley,15,6,49.088,40.0,3.27,0,2025-06-29,Rocket Classic
2,Aaron Rai,17,13,660.480,76.5,38.85,2,2025-06-29,Rocket Classic
3,Aaron Wise,1,0,0.000,0.0,0.00,0,2025-06-29,Rocket Classic
4,Adam Hadwin,18,12,212.382,66.7,11.80,3,2025-06-29,Rocket Classic
5,Adam Long,1,0,0.000,0.0,0.00,0,2025-06-29,Rocket Classic
6,Adam Schenk,19,9,270.904,47.4,14.26,2,2025-06-29,Rocket Classic
7,Adam Scott,13,11,392.203,84.6,30.17,6,2025-06-29,Rocket Classic
8,Adam Svensson,19,10,126.043,52.6,6.63,0,2025-06-29,Rocket Classic
9,Adrien Dumont de Chassart,7,3,200.500,42.9,28.64,0,2025-06-29,Rocket Classic


## Recent Form

In [22]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Carlos Ortiz,1,4.0,5.77,2025-06-29,Rocket Classic
1,Scottie Scheffler,15,6.9,2.49,2025-06-29,Rocket Classic
2,Jon Rahm,3,9.7,7.00,2025-06-29,Rocket Classic
3,Bubba Watson,1,14.0,20.20,2025-06-29,Rocket Classic
4,Ian Gilligan,1,16.0,23.08,2025-06-29,Rocket Classic
5,Taisei Shimizu,1,16.0,23.08,2025-06-29,Rocket Classic
6,Stewart Cink,1,17.0,24.53,2025-06-29,Rocket Classic
7,Rory McIlroy,11,19.0,7.65,2025-06-29,Rocket Classic
8,Ren Yonezawa,1,22.0,31.74,2025-06-29,Rocket Classic
9,Yuto Katsuragawa,1,22.0,31.74,2025-06-29,Rocket Classic


## Course History

In [23]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,4,73.3,45.54,2025-06-29,Detroit Golf Club,Rocket Classic
1,Aaron Rai,2,5.5,5.01,2025-06-29,Detroit Golf Club,Rocket Classic
2,Aaron Wise,2,62.5,56.89,2025-06-29,Detroit Golf Club,Rocket Classic
3,Adam Hadwin,3,32.0,23.08,2025-06-29,Detroit Golf Club,Rocket Classic
4,Adam Long,3,90.0,64.92,2025-06-29,Detroit Golf Club,Rocket Classic
5,Adam Schenk,5,42.0,23.44,2025-06-29,Detroit Golf Club,Rocket Classic
6,Adam Svensson,3,73.3,52.87,2025-06-29,Detroit Golf Club,Rocket Classic
7,Adrien Dumont de Chassart,1,90.0,129.84,2025-06-29,Detroit Golf Club,Rocket Classic
8,Akshay Bhatia,2,46.0,41.87,2025-06-29,Detroit Golf Club,Rocket Classic
9,Aldrich Potgieter,1,90.0,129.84,2025-06-29,Detroit Golf Club,Rocket Classic


## Merged Dataframe

In [24]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Baddeley,160.0,-1.022,168.0,-1.292,137.0,-0.265,4.0,0.536,153.0,-0.445,169.0,18.10%,103.0,3.08,151.0,4.08,169.0,4.77,167.0,270,157.0,289.6,113.0,57.29%,164.0,60.48%,81.0,60.24%,466.0,.2814,2025.0,2000.0,40.0,49.088,3.27,0.0,77.6,27.99,73.3,45.54,6100
1,Aaron Wise,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6200
2,Adam Hadwin,146.0,-0.523,136.0,-0.251,149.0,-0.371,62.0,0.100,114.0,-0.111,59.0,22.02%,29.0,3.02,136.0,4.06,136.0,4.67,91.0,173,103.0,298.7,70.0,60.42%,120.0,64.34%,159.0,53.82%,107.0,1.2058,2025.0,200.0,66.7,212.382,11.80,3.0,60.8,20.65,32.0,23.08,6800
3,Adam Schenk,115.0,-0.129,89.0,0.078,119.0,-0.126,117.0,-0.081,107.0,-0.056,119.0,20.49%,37.0,3.03,155.0,4.09,36.0,4.57,135.0,209,52.0,305.1,157.0,52.40%,159.0,61.23%,96.0,59.24%,151.0,.8787,2025.0,250.0,47.4,270.904,14.26,2.0,65.4,21.83,42.0,23.44,6700
4,Adam Svensson,77.0,0.119,119.0,-0.087,93.0,0.015,40.0,0.190,154.0,-0.467,155.0,19.27%,103.0,3.08,104.0,4.04,66.0,4.60,94.0,176,147.0,293.8,29.0,64.50%,50.0,66.78%,93.0,59.39%,198.0,.6987,2025.0,250.0,52.6,126.043,6.63,0.0,66.2,22.10,73.3,52.87,6700


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [25]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [26]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [27]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [28]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Baddeley,160.00000,-1.022000,168.000000,-1.292000,137.000000,-0.265000,4.000000,0.53600,153.000000,-0.445000,169.000000,18.100000,103.000000,3.080000,151.000000,4.080000,169.000000,4.770,167.000000,270.000000,157.000000,289.600000,113.000000,57.290000,164.00000,60.480000,81.00000,60.240000,466.0,0.2814,2025.0,1000.0,40.000000,49.088,3.270000,0.000000,77.6,27.990000,73.300000,45.540000,6100
1,Aaron Wise,93.62931,0.001655,88.724138,0.006319,92.862069,-0.005897,88.836207,0.00131,94.844828,-0.048741,92.577586,21.241379,80.956897,3.064052,83.594828,4.031121,82.508621,4.615,83.853448,169.146552,79.982759,301.984483,89.163793,59.192845,87.87069,65.341983,83.12069,59.902586,1000.0,6.0895,2025.0,1000.0,55.133088,0.000,20.951838,1.610294,90.0,22.503309,57.139423,53.776923,6200
2,Adam Hadwin,146.00000,-0.523000,136.000000,-0.251000,149.000000,-0.371000,62.000000,0.10000,114.000000,-0.111000,59.000000,22.020000,29.000000,3.020000,136.000000,4.060000,136.000000,4.670,91.000000,173.000000,103.000000,298.700000,70.000000,60.420000,120.00000,64.340000,159.00000,53.820000,107.0,1.2058,2025.0,200.0,66.700000,212.382,11.800000,3.000000,60.8,20.650000,32.000000,23.080000,6800
3,Adam Schenk,115.00000,-0.129000,89.000000,0.078000,119.000000,-0.126000,117.000000,-0.08100,107.000000,-0.056000,119.000000,20.490000,37.000000,3.030000,155.000000,4.090000,36.000000,4.570,135.000000,209.000000,52.000000,305.100000,157.000000,52.400000,159.00000,61.230000,96.00000,59.240000,151.0,0.8787,2025.0,250.0,47.400000,270.904,14.260000,2.000000,65.4,21.830000,42.000000,23.440000,6700
4,Adam Svensson,77.00000,0.119000,119.000000,-0.087000,93.000000,0.015000,40.000000,0.19000,154.000000,-0.467000,155.000000,19.270000,103.000000,3.080000,104.000000,4.040000,66.000000,4.600,94.000000,176.000000,147.000000,293.800000,29.000000,64.500000,50.00000,66.780000,93.00000,59.390000,198.0,0.6987,2025.0,250.0,52.600000,126.043,6.630000,0.000000,66.2,22.100000,73.300000,52.870000,6700
5,Akshay Bhatia,78.00000,0.115000,107.000000,-0.025000,24.000000,0.458000,152.000000,-0.31800,15.000000,0.422000,20.000000,23.710000,8.000000,2.990000,64.000000,4.020000,152.000000,4.700,135.000000,209.000000,147.000000,293.800000,62.000000,61.100000,55.00000,66.370000,157.00000,54.870000,36.0,2.5932,2025.0,45.0,76.500000,810.740,47.690000,1.000000,40.8,14.120000,46.000000,41.870000,8200
6,Aldrich Potgieter,111.00000,-0.094000,5.000000,0.637000,141.000000,-0.272000,161.000000,-0.45900,34.000000,0.252000,58.000000,22.060000,65.000000,3.050000,119.000000,4.050000,153.000000,4.710,53.000000,151.000000,1.000000,326.600000,150.000000,53.730000,138.00000,63.170000,162.00000,53.450000,123.0,1.1014,2025.0,125.0,36.400000,441.000,40.090000,1.000000,63.6,25.590000,90.000000,129.840000,7000
7,Alejandro Tosti,43.00000,0.540000,29.000000,0.335000,114.000000,-0.101000,18.000000,0.30600,164.000000,-0.681000,42.000000,22.460000,91.000000,3.070000,64.000000,4.020000,50.000000,4.580,30.000000,129.000000,10.000000,314.100000,119.000000,56.780000,113.00000,64.490000,71.00000,60.540000,146.0,0.9258,2025.0,200.0,44.400000,316.783,17.600000,0.000000,65.1,22.110000,90.000000,129.840000,6800
8,Alex Noren,93.62931,0.001655,88.724138,0.006319,92.862069,-0.005897,88.836207,0.00131,94.844828,-0.048741,92.577586,21.241379,80.956897,3.064052,83.594828,4.031121,82.508621,4.615,83.853448,169.146552,79.982759,301.984483,89.163793,59.192845,87.87069,65.341983,83.12069,59.902586,76.0,1.6565,2025.0,60.0,80.000000,134.250,26.850000,1.000000,45.4,25.340000,48.300000,30.010000,7800
9

## Correlations
**Last 4 Years**

In [29]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [30]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [31]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.9338784  0.95882353 0.9662854  0.96781046 0.91249309]
Average ROC AUC score: 0.95


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [34]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Max Greyserman,9100,0.836727,35.6,31.000000,40.0,85.0,48.0,517.887,83.00000,114.000000,98.000000,54.000000,41.000000,39.000000,117.000000,88.000000,66.000000,99.000000,36.000000,141.000000,107.00000,141.00000
1,Ben Griffin,9800,0.813217,35.6,32.000000,20.0,81.5,17.0,2039.496,26.00000,51.000000,34.000000,56.000000,35.000000,52.000000,29.000000,46.000000,36.000000,53.000000,64.000000,87.000000,17.00000,102.00000
2,Davis Thompson,8100,0.673072,50.3,28.000000,40.0,70.0,61.0,485.617,41.00000,27.000000,85.000000,46.000000,149.000000,37.000000,103.000000,88.000000,36.000000,24.000000,73.000000,41.000000,42.00000,124.00000
3,Wyndham Clark,8600,0.667525,45.1,17.000000,40.0,82.4,29.0,422.873,94.00000,66.000000,148.000000,36.000000,64.000000,28.000000,145.000000,88.000000,23.000000,42.000000,15.000000,129.000000,115.00000,134.00000
4,Ryan Gerard,7900,0.664504,35.3,56.000000,40.0,94.4,70.0,812.212,27.00000,49.000000,32.000000,65.000000,104.000000,90.000000,47.000000,64.000000,114.000000,35.000000,81.000000,58.000000,121.00000,77.00000
5,Tony Finau,8900,0.648425,45.3,71.500000,40.0,75.0,44.0,670.408,65.00000,107.000000,77.000000,31.000000,121.000000,50.000000,117.000000,104.000000,127.000000,89.000000,67.000000,105.000000,91.00000,130.00000
6,Alex Noren,7800,0.644483,45.4,48.300000,60.0,80.0,76.0,134.250,93.62931,88.724138,92.862069,88.836207,94.844828,92.577586,80.956897,83.594828,82.508621,83.853448,79.982759,89.163793,87.87069,83.12069
7,Byeong Hun An,7600,0.637630,50.0,59.700000,60.0,72.2,46.0,588.934,73.00000,34.000000,142.000000,41.000000,147.000000,87.000000,117.000000,64.000000,136.000000,102.000000,44.000000,135.000000,127.00000,110.00000
8,Jacob Bridgeman,7300,0.616447,49.8,31.000000,90.0,62.5,69.0,1135.114,130.00000,122.000000,111.000000,114.000000,7.000000,77.000000,47.000000,104.000000,66.000000,157.000000,117.000000,116.000000,136.00000,64.00000
9,Akshay Bhatia,8200,0.616323,40.8,46.000000,45.0,76.5,36.0,810.740,78.00000,107.000000,24.000000,152.000000,15.000000,20.000000,8.000000,64.000000,152.000000,135.000000,147.000000,62.000000,55.00000,157.00000


### Save to CSV

In [35]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
